<a href="https://colab.research.google.com/github/waltersbrf/Doutorado/blob/main/FAQ_com_Langchain_e_LLMs_OpenAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo do Notebook 

# Introdução

LangChain é um framework para desenvolvimento de aplicações usando Grandes Modelos de Linguagem (Large Language Model - LLMs). Com a combinação  das LLMs com outros ativos é possível gerar aplicações que envolvem gerenciamento de prompt, encadeamento (chain), geração de dados aumentados(data augmentation), designação de sequência de operações, etc.


No caso deste notebook, será apresentado um processo de criação de FAQ (Frequently Asked Questions Em português seria algo como "Perguntas frequentes") sobre um tema. No caso, o tema em questão envolve atores de um sistema de rastreio de satélites.


Para trabalhar num tema com LangChain, será necessário informar alguns ativos para servir de base para o processamento, no caso esse exemplo irá fornecer um conjunto de manuais e artigos sobre o tema. 

Como observação, as respostas devem variar de acordo com o material de suporte fornecido. 

Os esforços destes notebook focam na execução deste conjunto no Google colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "seu token aqui - OpenIA"

In [ ]:

pdf_folder_path = '/content/gdrive/MyDrive/data2'

# Instalando dependências 

In [ ]:
!pip install openai

!pip install langchain
!pip install sentence_transformers
!pip install faiss-cpu

!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]

!pip install pdf2image
!pip install pytesseract

!pip install PyPDF2

!apt-get install poppler-utils tesseract-ocr

# Implementando as Operações

In [ ]:
import os 

from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator


loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
loaders

In [ ]:
#from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
#from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
index = VectorstoreIndexCreator(
    embedding= OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=2500, chunk_overlap=0)).from_loaders(loaders)

In [ ]:

chain = RetrievalQA.from_chain_type(llm=OpenAI() ,
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")


In [ ]:
chain.run('Os textos falam sobre o que?')